# 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa
import warnings
import random
from sklearn.model_selection import train_test_split


warnings.filterwarnings("ignore")

# wav 데이터 처리하기

In [ ]:
sample_submission = pd.read_csv("./sample_submission.csv")

africa_train_paths = glob("./train/africa/*.wav")
australia_train_paths = glob("./train/australia/*.wav")
canada_train_paths = glob("./train/canada/*.wav")
england_train_paths = glob("./train/england/*.wav")
hongkong_train_paths = glob("./train/hongkong/*.wav")
us_train_paths = glob("./train/us/*.wav")

path_list = [africa_train_paths, australia_train_paths, canada_train_paths,
             england_train_paths, hongkong_train_paths, us_train_paths]

In [ ]:
def get_id(data):
    return np.int(data.split("/")[2].split(".")[0])


test_ = pd.DataFrame(index=range(0, 6100), columns=["path", "id"])
test_["path"] = glob("./test/*.wav")
test_["id"] = test_["path"].apply(lambda x: get_id(x))

test_.head()

In [ ]:
def load_data(paths):

    result = []
    for path in tqdm(paths):
        # sr = 16000이 의미하는 것은 1초당 16000개의 데이터를 샘플링 한다는 것입니다.
        data, sr = librosa.load(path, sr=16000)
        result.append(data)
    result = np.array(result)
    # 메모리가 부족할 때는 데이터 타입을 변경해 주세요 ex) np.array(data, dtype = np.float32)

    return result

In [ ]:
# train 데이터를 로드하기 위해서는 많은 시간이 소모 됩니다.
# 따라서 추출된 정보를 npy파일로 저장하여 필요 할 때마다 불러올 수 있게 준비합니다.


africa_train_data = load_data(africa_train_paths)
np.save("./npy_data/africa_npy", africa_train_data)

australia_train_data = load_data(australia_train_paths)
np.save("./npy_data/australia_npy", australia_train_data)

canada_train_data = load_data(canada_train_paths)
np.save("./npy_data/canada_npy", canada_train_data)

england_train_data = load_data(england_train_paths)
np.save("./npy_data/england_npy", england_train_data)

hongkong_train_data = load_data(hongkong_train_paths)
np.save("./npy_data/hongkong_npy", hongkong_train_data)

us_train_data = load_data(us_train_paths)
np.save("./npy_data/us_npy", us_train_data)

test_data = load_data(test_["path"])
np.save("./npy_data/test_npy", test_data)

In [ ]:
test_data = load_data(test_["path"])
np.save("./npy_data/test_npy", test_data)

In [ ]:
# npy파일로 저장된 데이터를 불러옵니다.
africa_train_data = np.load("./npy_data/africa_npy.npy", allow_pickle=True)
australia_train_data = np.load(
    "./npy_data/australia_npy.npy", allow_pickle=True)
canada_train_data = np.load("./npy_data/canada_npy.npy", allow_pickle=True)
england_train_data = np.load("./npy_data/england_npy.npy", allow_pickle=True)
hongkong_train_data = np.load("./npy_data/hongkong_npy.npy", allow_pickle=True)
us_train_data = np.load("./npy_data/us_npy.npy", allow_pickle=True)


train_data_list = [africa_train_data, australia_train_data[:1000], canada_train_data[:1000],
                   england_train_data[:2500], hongkong_train_data[:1000], us_train_data[:2500]]

In [ ]:
test_data = np.load("./npy_data/test_npy.npy", allow_pickle=True)

In [ ]:
# 이번 대회에서 음성은 각각 다른 길이를 갖고 있습니다.
# baseline 코드에서는 음성 중 길이가 가장 작은 길이의 데이터를 기준으로 데이터를 잘라서 사용합니다.

def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

# 음성들의 길이를 맞춰줍니다.


def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

# feature를 생성합니다.


def get_feature(data, sr=16000, n_fft=256, win_length=200, hop_length=160, n_mels=64):
    mel = []
    for i in data:
        # win_length 는 음성을 작은 조각으로 자를때 작은 조각의 크기입니다.
        # hop_length 는 음성을 작은 조각으로 자를때 자르는 간격을 의미합니다.
        # n_mels 는 적용할 mel filter의 개수입니다.
        mel_ = librosa.feature.melspectrogram(
            i, sr=sr, n_fft=n_fft, win_length=win_length, hop_length=hop_length, n_mels=n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref=np.max)

    mel_mean = mel.mean()
    mel_std = mel.std()
    mel = (mel - mel_mean) / mel_std

    return mel

In [ ]:
train_x = np.concatenate(train_data_list, axis=0)
test_x = np.array(test_data)

# 음성의 길이 중 가장 작은 길이를 구합니다.

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

mini = np.min([train_mini, test_mini])

# data의 길이를 가장 작은 길이에 맞춰 잘라줍니다.

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

# librosa를 이용해 feature를 추출합니다.

train_x = get_feature(data=train_x)
test_x = get_feature(data=test_x)

train_x = train_x.reshape(-1, train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)

In [ ]:
# train_data의 label을 생성해 줍니다.

train_y = np.concatenate((np.zeros(2500, dtype=np.int),
                          np.ones(1000, dtype=np.int),
                          np.ones(1000, dtype=np.int) * 2,
                          np.ones(2500, dtype=np.int) * 3,
                          np.ones(1000, dtype=np.int) * 4,
                          np.ones(2500, dtype=np.int) * 5), axis=0)

In [ ]:
train_x.shape, train_y.shape, test_x.shape

# 모델링

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms

In [ ]:
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""

    def __call__(self, sample):
        x, y = sample['x'], sample['y']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        x = x.transpose((2, 0, 1))
        return {'x': torch.FloatTensor(x),
                'y': torch.FloatTensor(y)}


to_tensor = transforms.Compose([
    ToTensor()
])


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, train_x, train_y, transforms=to_tensor):
        self.x_data = train_x
        self.y_data = train_y
        self.transforms = transforms  # Transform

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):

        x = self.x_data[idx]
        y = self.y_data[idx]
        sample = {'x': x, 'y': y}
        if self.transforms:
            sample = self.transforms(sample)
        y = y.astype(np.float32)

        return (x, y)

In [ ]:
def get_split_data(X, y):
    """
    train 데이터를 8:2 비율로 train과 valid 데이터로 나누는 함수

    """
    X_len = len(X)
    idx = list(range(X_len))

    train_idx, valid_idx, _, _ = train_test_split(
        idx, y, stratify=y, test_size=0.2)

    train_x, train_y = X[train_idx], y[train_idx]
    valid_x, valid_y = X[valid_idx], y[valid_idx]

    return train_x, train_y, valid_x, valid_y


def get_dataloader(X, y, mode, batch_size):
    """
    데이터프레임을 dataloader형태로 반환하는 함수
    """

    if mode == 'TRAIN':
        train_x, train_y, valid_x, valid_y = get_split_data(X, y)

        # Data Loader
        train_dataset = CustomDataset(train_x, train_y)
        valid_dataset = CustomDataset(valid_x, valid_y)

        train_data_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=3
        )
        valid_data_loader = DataLoader(
            valid_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=3
        )
        return train_data_loader, valid_data_loader

    else:
        test_dataset = CustomDataset(X, y)
        test_data_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=3,
            drop_last=False
        )
        return test_data_loader

# 모델링

In [ ]:
class MultiLabeleffnet(nn.Module):
    def __init__(self):
        super(MultiLabeleffnet, self).__init__()
        self.conv2d = nn.Conv2d(64, 3, 1, stride=1)
        self.resnet = models.efficientnet_b7()
        self.FC = nn.Linear(1000, 6)

    def forward(self, x):
        # resnet의 입력은 [3, N, N]으로
        # 3개의 채널을 갖기 때문에
        # resnet 입력 전에 conv2d를 한 층 추가
        x = F.silu(self.conv2d(x))

        # resnet18을 추가
        x = F.silu(self.resnet(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        #x = self.FC(x)
        x = self.FC(x)
        return x
# 모델 선언

#model = MultiLabeleffnet()
# model.to(device)

In [ ]:
def train_model(model, dataloaders_dict, optimizer, num_epochs, device):
    """
    train 데이터로 모델을 학습하고 valid 데이터로 모델을 검증하는 코드

    파라미터
    ---
    model : 
        학습할 모델
    dataloaders_dict : dict
        train_dataloader과 validation_datalodaer가 들어 있는 dictonary
    optimizer : 
        최적화 함수
    num_epochs : int
        학습 횟수
    device : cuda or cpu
        모델을 학습할 때 사용할 장비

    returns 
    best_model :
        검증데이터 셋 기준으로 가장 성능이 좋은 모델
    ---
    """
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=5,
                                                   gamma=0.9)
    model.to(device)
    torch.cuda.empty_cache()
    critrion = torch.nn.CrossEntropyLoss()
    # 학습이 어느정도 진행되면 gpu 가속화
    #torch.backends.cudnn.benchmark = False

    # loss가 제일 낮은 모델을 찾기위한 변수
    best_val_loss = int(1e9)
    best_model = model
    for epoch in range(num_epochs):
        # epoch 별 학습 및 검증

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로
            else:
                model.eval()   # 모델을 추론 모드로

            epoch_loss = 0.0  # epoch loss
            epoch_corrects = 0  # epoch 정확도
            for i,  batch in enumerate(dataloaders_dict[phase]):
                images, labels = batch
                # tensor를 gpu에 올리기
                images = images.to(device)
                labels = labels.to(device)

                # 옵티마이저 초기화 초기화
                optimizer.zero_grad()

                # 순전파 계산
                with torch.set_grad_enabled(phase == 'train'):
                    probs = model(images)

                    loss = critrion(probs, labels.long())

                    # 학습시 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    # loss계산
                    epoch_loss += loss.item()
                    # 정확도 계산
                    # train accuracy 계산
                    probs = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    batch_acc = (labels == np.argmax(probs, axis=1)).mean()
                    epoch_corrects += batch_acc
            # epoch별 loss 및 정확도

            epoch_loss = epoch_loss / len(dataloaders_dict[phase])
            epoch_acc = 100 * epoch_corrects / \
                len(dataloaders_dict[phase])

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

            # 검증 오차가 가장 적은 최적의 모델을 저장
            if phase == 'valid' and epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                best_model = model

            lr_scheduler.step()

    return best_model

In [ ]:
best_models = []
# 교차 검증을 진행할 K
K = 1
for i in range(K):

    train_loader, val_loader = get_dataloader(
        train_x, train_y, mode='TRAIN', batch_size=256)
    #  dict 형식으로 data loader 정의
    dataloaders_dict = {"train": train_loader, "val": val_loader}
    num_epochs = 100
    max_grad_norm = 1
    learning_rate = 5e-5

    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    print('사용하는 device :', device)
    print('-----------------', i+1, '/', K,
          '- fold start-----------------')

    model = MultiLabeleffnet()

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    best_model = train_model(model, dataloaders_dict,
                             optimizer, num_epochs, device)
    best_models.append(best_model)

In [ ]:
def inference(test_loader, model):
    pred_ = []
    for idx, sample in enumerate(test_loader):
        with torch.no_grad():
            # 추론
            model.eval()
            images, _ = sample
            images = images.to(device)
            probs = model(images)
            probs = F.softmax(probs)
            probs = probs.cpu().detach().numpy()
            pred_.append(probs)
    return pred_

In [ ]:
# fold 별 best model을 이용하여 inference 진행
# 이후 결과를 hard voting 진행
preds = []
for best_model in best_models:

    test_y = np.array(range(len(test_x)))
    test_loader = get_dataloader(
        test_x, test_y, mode='TEST', batch_size=256)
    preds.append(inference(test_loader, best_model))

In [ ]:
final_pred = np.argmax(preds,axis = 1)

In [ ]:
def cov_type(data):
    return np.int(data)


# 처음에 살펴본 것처럼 glob로 test data의 path는 sample_submission의 id와 같이 1,2,3,4,5.....으로 정렬 되어있지 않습니다.
# 만들어둔 test_ 데이터프레임을 이용하여 sample_submission과 predict값의 id를 맞춰줍니다.
sample_submission = pd.read_csv("./open/sample_submission.csv")
result = pd.concat([test_, pd.DataFrame(
    np.mean(final_pred, axis=0))], axis=1).iloc[:, 1:]
result["id"] = result["id"].apply(lambda x: cov_type(x))

result = pd.merge(sample_submission["id"], result)
result.columns = sample_submission.columns

In [ ]:
result.to_csv("submission.csv", index = False)